### 정적 페이지 크롤링하기 - 공공데이터

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
filename = "forecast.xml"
req.urlretrieve(url, filename)

# 저장된 xml 파일을 읽어들여, 데이터 추출
info = {}
xml = open(filename, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, 'xml')

# find_all("location") : <location> 태그들을 읽어옴
for location in soup.find_all("location"):
    city = location.find('city').string     # 도시
    min_t = location.find('tmn')            # 최저기온
    max_t = location.find('tmx')            # 최고기온
    date = location.find_all('tmEf')        # 날짜시간
    # 최저기온~최기온 (10~25)
    # weather = [ a.string+"~"+b.string for a, b in zip(min_t, max_t) ]
    # print(f'{city} : {min_t} ~ {max_t}')
    # print(weather)
    weather ={}
    # print( len(min_t) )
    # print( len(max_t) )
    # print( len(date) )
    for i in range( len(min_t) ):           # 13 
        item = {}
        item['min_t'] = min_t[i].string
        item['max_t'] = max_t[i].string
        item['date'] = date[i].string
        weather.append(item)
    
    # print(city)
    # print(weather)
    
    if not (city in info):
        info[city] = []             # {'서울' : ['22~29']}
    for data in weather:
        info[city].append(data)     # {'서울' : ['22~29', '22~29', '23~31', '23~31', ...]}
    
    print(info)
    
    

ModuleNotFoundError: No module named 'bs4'

In [2]:
import urllib.request as req

# 버스번호로 busRouteId 가져오기
busNum = '6628'
key = ''
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum'
res = req.urlopen(url1)

print(res.read().decode('utf-8'))

soup = BeautifulSoup( result, 'xml' )

busRouteId = None
for itemList in soup.find_all('itemList'):
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break
    
print(f' busRoute Id : {busRouteId}')
print(f' busRoute Nm : {busRouteNm}')

# 노선별 경유 정류소 조회 서빗

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?serviceKey='+key+'&busRouteId='+busRouteId'
res = req.urlopen(url2)
result = res.read()
print(result)
soup = BeautifulSoup( result, 'xml' )

# print( soup.prettify() )

busPos = []
for itemList in soup.find_all('itemList'):
    stationNm = itemList.find('stationNm')
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string
    beginTm = itemList.find('beginTm').string
    lastTm = itemList.find('lastTm').string

    busPos.append( (stationNm, gpsY, gpsX, beginTm, lastTm))
    
    print( busNum + '번 버스 정류소 위치')
    for stationNm, lat, lng, beginTm, lastTm in busPos:
        print( stationNm, ':', lat + ',' + lng + ' (' + beginTm + '~' + lastTm + ')')     # 위도, 경도
        


SyntaxError: unterminated string literal (detected at line 6) (2561039517.py, line 6)

### 네이버 블로그 검색

In [5]:
import requests
import urllib
import re

# 네이버 애플리케이션 등록 - client_id, client_secret
client_id = 'HSTv_KSkNfyDlXFM1ENf'
client_secret = '9b4eXJyCQ3'

query = '영등포 맛집'
query = urllib.parse.quote_plus(query)
display = str(20)

url = 'https://openapi.naver.com/v1/search/blog.json?query=' + query + '&display=' + display
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret}

res = requests.get( url, headers=headers )
status = res.status_code

if( status == 200):
    data_result = res.json()
    # print(data_result)
    for count, data in enumerate(data_result['items'], 1):
        title = re.sub("</?b>", "", data['title'])                  # 제목에서 <b> </b> 제거
        description = re.sub("</?b>", "", data['description'])      # 내용에서 <b> </b> 제거
        print(str(count) + ' : ' + title)
        print('--------------------------')
        print(description)
        print('블로그명 : ' + data['bloggername'])
        print( 'url : ' + data['link'])
        
    # print( data_result )
    
    else:
        print('요청 실패 : ' + status)

1 : 영등포 맛집 &lt;남영동양문 영등포점&gt; - 서비스 좋은 영등포 고기집
--------------------------
소고기를 합리적인 가격에 먹을 수 있어서 만족스러웠던 &lt;남영동양문 영등포점&gt; ^_^ 영등포 맛집 &lt;남영동양문 영등포점&gt; 영업시간 매일 14:00 ~ 24:00 연락처 0507-1450-0311 &lt;남영동양문 영등포점&gt;의 네이버플레이스... 
블로그명 : 새롱일기
url : https://blog.naver.com/srkang1212/223131403434
2 : 영등포 타임스퀘어 맛집 띤띤: 영등포 베트남음식 쌀국수의 반란
--------------------------
베트남 현지의 맛을 느끼고싶다면 영등포 베트남음식으로 이미 인정받고 있는 영등포 타임스퀘어 맛집 띤띤 추천! #영등포타임스퀘어맛집 #영등포베트남음식 #띤띤 #영등포쌀국수 #영등포맛집 #영등포역쌀국수
블로그명 : 우리들의 블록스
url : https://blog.naver.com/ahmiun/223125981181
3 : 영등포구청 카페 필댐굿 영등포 맛집 인정! (feat. 주차장 팁)
--------------------------
이 집은 크루아상이 유명한 맛집이예요. 영등포구청 근처에서 가장 핫한 크루아상 가게라고 생각하시면... 영등포 맛집이니 꼭 들러보시기 바래요. 바질페스토, 크림치즈, 토마토 등이 들어기 있는 치아바타... 
블로그명 : it Travel
url : https://blog.naver.com/ittravel/223104521992
4 : 당산 맛집 해심 영등포구청 근처 핫플 횟집으로 떠오른... 
--------------------------
깔끔하고 깨끗하게 먹을 수 있는 식당이라 데이트하기에도 좋을 것 같아요. &lt;본 포스팅은 소정의 원고료를 받고 작성되었습니다&gt; #해심당산본점 #영등포맛집 #당산맛집 #영등포구청역맛집 #영등포횟집 #당산횟집
블로그명 : 맛집, 추천합니다
url : https:

TypeError: can only concatenate str (not "int") to str

### 네이버 뉴스 기사 검색

In [9]:
import requests
import urllib
import re
import json

# 네이버 애플리케이션 등록 - client_id, client_secret
client_id = 'HSTv_KSkNfyDlXFM1ENf'
client_secret = '9b4eXJyCQ3'

query = '올해의 '
query = urllib.parse.quote_plus(query)
display = str(20)

url = 'https://openapi.naver.com/v1/search/news.json?query=' + query + '&display=' + display
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret}

request = urllib.request.Request(url)
request.add_header('X-Naver-Client-Id', client_id)
request.add_header('X-Naver-Client-Secret', client_secret)

res = urllib.request.urlopen(request)
status = res.getcode()

if( status == 200):
    result = res.read()
    result = json.loads(result)
    
    count = 1
    for data in result['items']:
        print( str(count) + ' : ' + data['title'])
        print('====================================')
        print(': ' + data['description'])
        count += 1
    # print(result)

    else:
        print('요청 실패 : ' + status)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1002)>